<a href="https://colab.research.google.com/github/tribecca37/NYC_Jobs/blob/main/nycjob0514_extracttoStaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.6 MB/s eta 0:00:00


In [ ]:
# import Librairies
import pandas as pd
import numpy as  np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO

In [ ]:
# Function
import os
import boto3
from azure.storage.blob import BlobServiceClient
from google.cloud import storage
import pandas as pd
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()



s1 --> Gather --> Deccompress --> Convert to Tabular --> Clean  --> Reformat ---> Consolidation --> Transformation --> Load

In [11]:
import requests
response = requests.get("https://data.cityofnewyork.us/api/views/kpav-sd4t/rows.json?accessType=DOWNLOAD")
print(response.status_code)

200


In [16]:
import requests
import pandas as pd

# API endpoint URL
url = "https://data.cityofnewyork.us/api/views/kpav-sd4t/rows.json?accessType=DOWNLOAD"

# Sending a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()  # Convert JSON response to a dictionary
    # Assuming the actual data is under the 'data' key and column names under 'columns'
    columns = [col['name'] for col in data['meta']['view']['columns']]
    df_raw = pd.DataFrame(data['data'], columns=columns)
    print(df_raw.head())
    print(df_raw.columns)
    # Access only specific columns, check if they exist first:
    column_subset = ['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title']
    if set(column_subset).issubset(df_raw.columns):
        selected_df = df_raw[column_subset]
        print(selected_df.head())
    else:
        print("Some of the specified columns are missing in the DataFrame.")
    print(df_raw.shape)  # Print the shape of the DataFrame
else:
    print("Failed to fetch data:", response.status_code)
    print(response.text)

                  sid                                    id  position  \
0  row-srwb~tx2x~n8ev  00000000-0000-0000-4285-5E750C26AC67         0   
1  row-gmmp~st9n.mrhz  00000000-0000-0000-4E2B-B9D7C965EA43         0   
2  row-7ijt_sqje-vi6i  00000000-0000-0000-D645-BE58A56A3049         0   
3  row-zgeu~yra9-82sy  00000000-0000-0000-9A4B-BE8620848B21         0   
4  row-atvs.3bjf~45tx  00000000-0000-0000-D663-1B517329E940         0   

   created_at created_meta  updated_at updated_meta meta  Job ID  \
0  1715108614         None  1715108614         None  { }  534519   
1  1715108614         None  1715108614         None  { }  631363   
2  1715108614         None  1715108614         None  { }  628158   
3  1715108614         None  1715108614         None  { }  629002   
4  1715108614         None  1715108614         None  { }  613253   

                           Agency  ...  \
0  DEPT OF ENVIRONMENT PROTECTION  ...   
1  DEPT OF ENVIRONMENT PROTECTION  ...   
2         BRONX DISTRICT A

In [17]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5069 entries, 0 to 5068
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   sid                            5069 non-null   object
 1   id                             5069 non-null   object
 2   position                       5069 non-null   int64 
 3   created_at                     5069 non-null   int64 
 4   created_meta                   0 non-null      object
 5   updated_at                     5069 non-null   int64 
 6   updated_meta                   0 non-null      object
 7   meta                           5069 non-null   object
 8   Job ID                         5069 non-null   object
 9   Agency                         5069 non-null   object
 10  Posting Type                   5069 non-null   object
 11  # Of Positions                 5069 non-null   object
 12  Business Title                 5069 non-null   object
 13  Civ

In [18]:
df_cleaned = df_raw.copy()

In [21]:
df_cleaned = df_raw.drop(columns=  ['created_meta', 'updated_meta', 'Recruitment Contact',  'Work Location 1', 'Hours/Shift','Post Until', 'Additional Information'])
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5069 entries, 0 to 5068
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   sid                            5069 non-null   object
 1   id                             5069 non-null   object
 2   position                       5069 non-null   int64 
 3   created_at                     5069 non-null   int64 
 4   updated_at                     5069 non-null   int64 
 5   meta                           5069 non-null   object
 6   Job ID                         5069 non-null   object
 7   Agency                         5069 non-null   object
 8   Posting Type                   5069 non-null   object
 9   # Of Positions                 5069 non-null   object
 10  Business Title                 5069 non-null   object
 11  Civil Service Title            5069 non-null   object
 12  Title Classification           5069 non-null   object
 13  Tit

In [22]:
CONNECTION_STRING_AZURE_STORAGE = "DefaultEndpointsProtocol=https;AccountName=nycjobs;AccountKey=IuQ3EcwNFwEaSl4kBB7pn6YFkrmux0GxuaqplhfQErilu32k8n7p4lw9ckn65O3VMTvGrUijAgaY+ASt6+1LYA==;EndpointSuffix=core.windows.net"
CONTAINER_AZURE = 'nycjobs'
blob_name = "nycjobs.csv"
# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")


Uploaded nycjobs.csv to Azure Blob Storage in container nycjobs.


In [24]:
# Example usage
if __name__ == "__main__":

    output = StringIO()
    df_raw.to_csv(output, index=False)
    data = output.getvalue()
    output.close()

    # Azure
    azure_connect_str = 'DefaultEndpointsProtocol=https;AccountName=nycjobs;AccountKey=IuQ3EcwNFwEaSl4kBB7pn6YFkrmux0GxuaqplhfQErilu32k8n7p4lw9ckn65O3VMTvGrUijAgaY+ASt6+1LYA==;EndpointSuffix=core.windows.net'
    azure_container_name = 'nycjobs'
    azure_blob_name = 'nycjobs.csv'
    # Upload CSV data to Azure Blob Storage
    azure_upload_blob(azure_connect_str, azure_container_name, azure_blob_name, data)

    # Download and print the blob content
    azure_blob_content = azure_download_blob(azure_connect_str, azure_container_name, azure_blob_name)
    print(azure_blob_content)


Uploaded to Azure Blob: nycjobs.csv


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

